# Demo code to train a regression model on microstructure data

Solution to a regression problem to solve for the kappa coefficient.

In [ ]:
# Load up packages
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import xarray as xr
import pandas as pd
from sklearn.tree import  DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
import sys
import pickle
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams.update({'font.size': 14})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
%cd /content/drive/MyDrive/randomforest

In [ ]:
# Load in data to a pandas dataframe
df = pd.read_pickle('input_microstructure.pkl')
df.reset_index(drop=True, inplace=True)

In [ ]:
df

,T,S,z,gamma,dTdz,dSdz,lat,lon,log_eps,log_kappa,log_N2_sort,hab,depth,cruise,profile
0,0.594316,34.733891,-3835.0,28.214811,0.002297,0.000235,-25.002001,-38.185333,-10.260384,-5.217481,-5.741873,93.0,-3928.0,1,0
1,0.621936,34.736645,-3825.0,28.212467,0.002297,0.000235,-25.002001,-38.185333,-10.004366,-4.961463,-5.741873,103.0,-3928.0,1,0
2,0.640263,34.738594,-3815.0,28.211014,0.001747,0.000191,-25.002001,-38.185333,-9.833233,-4.638316,-5.893887,113.0,-3928.0,1,0
3,0.656876,34.740475,-3805.0,28.209791,0.001718,0.000172,-25.002001,-38.185333,-10.048473,-4.886051,-5.861392,123.0,-3928.0,1,0
4,0.674622,34.742039,-3795.0,28.208130,0.002518,0.000253,-25.002001,-38.185333,-9.185361,-4.187189,-5.697142,133.0,-3928.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188128,1.588848,34.706534,-2815.0,28.042277,0.000482,-0.000054,-55.144500,-57.980500,-8.865618,-3.598640,-6.403454,160.0,-2975.0,14,170572
188129,1.584602,34.706882,-2825.0,28.043177,0.000509,-0.000039,-55.144500,-57.980500,-8.516571,-3.015310,-6.270092,150.0,-2975.0,14,170572
188130,1.578658,34.707318,-2835.0,28.044395,0.000321,0.000019,-55.144500,-57.980500,-8.574194,-2.781543,-6.177027,140.0,-2975.0,14,170572
188131,1.578185,34.706502,-2845.0,28.043852,0.000007,0.000055,-55.144500,-57.980500,-8.492338,-2.771025,-6.140031,130.0,-2975.0,14,170572


In [ ]:
# notice how mean values are far from the standard deviation.
# need to normalise data or choose an algorithm that deals with this inherently.
df.describe().T

,count,mean,std,min,25%,50%,75%,max
T,188133.0,4.782284,4.696371,-1.576699,2.034851,3.055532,5.539974,29.207123
S,188133.0,34.858130,0.504977,30.003132,34.624641,34.776505,34.935741,37.608685
z,188133.0,-1775.658205,1239.841793,-5525.000000,-2655.000000,-1535.000000,-735.000000,-5.000000
gamma,188133.0,27.644539,0.644622,19.231173,27.425438,27.883224,28.053850,28.294104
dTdz,188133.0,0.006038,0.013308,-0.153363,0.000603,0.001233,0.005223,0.530904
dSdz,188133.0,0.000330,0.002351,-0.306804,-0.000160,0.000019,0.000203,0.102666
lat,188133.0,-14.649828,32.073004,-64.344500,-40.000000,-21.540501,24.334999,37.396500
lon,188133.0,-43.509351,46.557865,-167.283340,-77.305336,-29.120333,-18.501833,74.787498
log_eps,188133.0,-9.693178,0.694475,-12.565783,-10.170434,-9.834250,-9.344311,-4.550475
log_kappa,188133.0,-4.834630,0.753433,-7.863402,-5.323518,-4.926121,-4.481439,-0.016134


import seaborn as sns
g = sns.pairplot(df, hue='log_kappa')
g.fig.suptitle("Scatterplot and histogram of pairs of variables color coded by risk level", 
               fontsize = 14, # defining the size of the title
               y=1.05); # y = definig title y position (height)

In [ ]:
# Features:
xstringlist = ['S','T','lat','hab','z','dSdz','dTdz','log_N2_sort']

# We want to predict:
ystringlist = ['log_kappa']

In [ ]:
#split into x and y, x has features, y is target
x = df[xstringlist]
y = df[ystringlist]

In [ ]:
from sklearn.model_selection import train_test_split

SEED = 42
X_train, X_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.2, 
                                                    random_state=SEED)

In [ ]:
X_train, X_test, y_train, y_test = X_train.to_numpy(dtype="float"), X_test.to_numpy(dtype="float"), y_train.to_numpy(dtype="float"), y_test.to_numpy(dtype="float")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

rfc = RandomForestClassifier(n_estimators=3, 
                             max_depth=2,
                             random_state=SEED)

rfr = RandomForestRegressor(random_state = 42)

In [ ]:
# Fit RandomForestClassifier
rfr.fit(X_train, y_train)
# Predict the test set labels
y_pred = rfr.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.6784850217550381

## Do random search for random forest regressor and hyperparameter tuning
From scipy documentation, max_features (nr of features considered when splitting) and n_estimators (number of trees in the forest) are most important.


In [ ]:
pd.Series(rfr.get_params())

bootstrap                            True
ccp_alpha                             0.0
criterion                   squared_error
max_depth                            None
max_features                          1.0
max_leaf_nodes                       None
max_samples                          None
min_impurity_decrease                 0.0
min_samples_leaf                        1
min_samples_split                       2
min_weight_fraction_leaf              0.0
n_estimators                          100
n_jobs                               None
oob_score                           False
random_state                           42
verbose                                 0
warm_start                          False
dtype: object

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)


{'n_estimators': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
rf_random = RandomizedSearchCV(estimator = rfr, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [ ]:
rf_random.best_params_

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, test_features, test_labels)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

## Do grid search for random forest regressor and hyperparameter tuning
From scipy documentation, max_features (nr of features considered when splitting) and n_estimators (number of trees in the forest) are most important.

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

## XGBRegressor from LOIS

In [ ]:
# Choose our regressor: other options are imported above. This is XGBoost, but can also try standard Random Forest, Decision Tree, etc. 
reg = XGBRegressor()

# Look at parameters of XGBRegressor to tune the model, e.g. reg = XGBRegressor(n_estimators=400, max_depth=10, eta = 0.1)

In [ ]:
# Perform a Kfold cross validation to see how well this model does:
cv = KFold(10,shuffle=True)
scores = cross_val_score(reg, x, y, cv=cv, scoring='r2')
print(scores)
print(scores.mean())

In [ ]:
# We can also check how this model does on the training and test data by splitting the data manually. Helps us see how well the model is overfitting. 

# Initialise r2 scores on train and test data
r2scores_train = []
r2scores_test = []

# Choose how many splits of the data to average over
n = 3

for i in tqdm(range(n)):
    dftrain, dftest = train_test_split(df,test_size=0.2,shuffle=True)
    
    # split into training x and y, x has features, y is target
    xtrain = dftrain[xstringlist]
    ytrain = dftrain[ystringlist]

    # split into testing x and y, x has features, y is target
    xtest = dftest[xstringlist]
    ytest = dftest[ystringlist]
    
    # fit model based on train data
    reg_fitted = reg.fit(xtrain, ytrain)
    
    # predict y based on test and train data
    ytestmodel = reg_fitted.predict(xtest)
    ytrainmodel = reg_fitted.predict(xtrain)
    
    r2scores_test.append(r2_score(ytest, ytestmodel))
    r2scores_train.append(r2_score(ytrain, ytrainmodel))
    

In [ ]:
# We ultimately want r2scores_test to be as high as possible (1 is perfect)
# but if r2scores_train is much higher, this suggests that the model is overfitting, 
# which means the model will generalise less well to data it hasn't seen 
print(np.mean(r2scores_test))
print(np.mean(r2scores_train))                  